# Carregar Dados de Exemplo

Este notebook carrega os dados de exemplo CSV para o MinIO (camada Bronze) para testes do motor de correspondência.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

# Caminho local dos arquivos CSV
PATH_DADOS_LOCAL = "./dados-exemplo/bronze"

# Lista de arquivos para carregar
arquivos_csv = {
    "enderecos_livres": "enderecos_livres.csv",
    "enderecos_com_erros": "enderecos_com_erros.csv",
    "dne": "dne.csv",
    "cnefe": "cnefe.csv",
    "osm": "osm.csv",
    "dados_geograficos": "dados_geograficos.csv",
    "variacoes_enderecos": "variacoes_enderecos.csv"
}

print("Arquivos CSV para carregar:")
for nome, arquivo in arquivos_csv.items():
    print(f"  {nome}: {arquivo}")

In [ ]:
# Função para carregar CSV e salvar no MinIO
def carregar_csv_para_minio(nome_tabela, arquivo_csv, destino_path):
    """
    Carrega um arquivo CSV local e salva no MinIO como Delta Table
    
    Args:
        nome_tabela: Nome da tabela
        arquivo_csv: Nome do arquivo CSV
        destino_path: Caminho de destino no MinIO
    """
    caminho_local = f"{PATH_DADOS_LOCAL}/{arquivo_csv}"
    
    print(f"\nCarregando {nome_tabela}...")
    print(f"  Arquivo: {caminho_local}")
    
    # Verificar se arquivo existe
    if not os.path.exists(caminho_local):
        print(f"  ERRO: Arquivo não encontrado: {caminho_local}")
        return None
    
    # Ler CSV
    df = spark.read.csv(
        caminho_local,
        header=True,
        inferSchema=True,
        encoding="utf-8"
    )
    
    print(f"  Registros carregados: {df.count()}")
    
    # Adicionar metadados
    df_com_metadados = df \
        .withColumn("carregado_em", current_timestamp()) \
        .withColumn("fonte_dados", lit("EXEMPLO"))
    
    # Salvar no MinIO
    print(f"  Salvando em: {destino_path}")
    save_delta_table(
        df_com_metadados,
        destino_path,
        mode="overwrite"
    )
    
    print(f"  ✓ {nome_tabela} carregado com sucesso!")
    
    return df_com_metadados

In [ ]:
# Carregar endereços livres
df_enderecos_livres = carregar_csv_para_minio(
    "enderecos_livres",
    arquivos_csv["enderecos_livres"],
    PATH_ENDERECOS_LIVRES
)

if df_enderecos_livres:
    df_enderecos_livres.show(5, truncate=False)

In [ ]:
# Carregar endereços com erros
df_enderecos_erros = carregar_csv_para_minio(
    "enderecos_com_erros",
    arquivos_csv["enderecos_com_erros"],
    f"{PATH_BRONZE}/enderecos_com_erros"
)

if df_enderecos_erros:
    df_enderecos_erros.show(5, truncate=False)

In [ ]:
# Carregar DNE
df_dne = carregar_csv_para_minio(
    "dne",
    arquivos_csv["dne"],
    PATH_DNE
)

if df_dne:
    df_dne.show(5, truncate=False)

In [ ]:
# Carregar CNEFE
df_cnefe = carregar_csv_para_minio(
    "cnefe",
    arquivos_csv["cnefe"],
    PATH_CNEFE
)

if df_cnefe:
    df_cnefe.show(5, truncate=False)

In [ ]:
# Carregar OSM
df_osm = carregar_csv_para_minio(
    "osm",
    arquivos_csv["osm"],
    PATH_OSM
)

if df_osm:
    df_osm.show(5, truncate=False)

In [ ]:
# Carregar dados geográficos
df_geograficos = carregar_csv_para_minio(
    "dados_geograficos",
    arquivos_csv["dados_geograficos"],
    f"{PATH_BRONZE}/dados_geograficos"
)

if df_geograficos:
    df_geograficos.show(5, truncate=False)

In [ ]:
# Carregar variações de endereços
df_variacoes = carregar_csv_para_minio(
    "variacoes_enderecos",
    arquivos_csv["variacoes_enderecos"],
    f"{PATH_BRONZE}/variacoes_enderecos"
)

if df_variacoes:
    df_variacoes.show(5, truncate=False)

In [ ]:
# Resumo de dados carregados
print("\n" + "="*60)
print("RESUMO DE DADOS CARREGADOS")
print("="*60)

tabelas = [
    ("Endereços Livres", PATH_ENDERECOS_LIVRES),
    ("Endereços com Erros", f"{PATH_BRONZE}/enderecos_com_erros"),
    ("DNE", PATH_DNE),
    ("CNEFE", PATH_CNEFE),
    ("OSM", PATH_OSM),
    ("Dados Geográficos", f"{PATH_BRONZE}/dados_geograficos"),
    ("Variações", f"{PATH_BRONZE}/variacoes_enderecos")
]

for nome, caminho in tabelas:
    try:
        df = read_delta_table(caminho)
        count = df.count()
        print(f"{nome:30} {count:>10} registros")
    except Exception as e:
        print(f"{nome:30} {'ERRO':>10} - {str(e)[:50]}")

print("="*60)
print("\nDados de exemplo carregados com sucesso!")
print("\nPróximos passos:")
print("  1. Executar notebook 01_tratamento_inicial_ner.ipynb")
print("  2. Executar notebook 02_normalizacao_camada_prata.ipynb")
print("  3. Executar notebook 03_camada_ouro_deduplicacao.ipynb")
print("  4. Executar notebook 05_motor_correspondencia.ipynb")
print("  5. Executar notebook 06_validacao_geografica.ipynb")